In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import scipy
import sklearn as sk
from sklearn.linear_model import LinearRegression

In [2]:
dfwind = pd.read_csv(filepath_or_buffer='wind/wind.csv')


dfwind.head()

,timestamp,wind_speed_ms-1,temperature
0,2015-01-01 00:00:00,10.48,26.0
1,2015-01-01 01:00:00,10.40,26.1
2,2015-01-01 02:00:00,10.53,26.2
3,2015-01-01 03:00:00,10.52,26.2
4,2015-01-01 04:00:00,10.37,26.1


In [4]:
#Esta parte del codigo cambia los valores que sean =< 27 y >= 7 por 0
dfwind.loc[dfwind['temperature'] >= 27,'wind_speed_ms-1'] = 0
dfwind.loc[dfwind['temperature'] <= 10,'wind_speed_ms-1'] = 0
dfwind

,timestamp,wind_speed_ms-1,temperature
0,2015-01-01 00:00:00,10.48,26.0
1,2015-01-01 01:00:00,10.40,26.1
2,2015-01-01 02:00:00,10.53,26.2
3,2015-01-01 03:00:00,10.52,26.2
4,2015-01-01 04:00:00,10.37,26.1
...,...,...,...
26299,2017-12-31 19:00:00,7.44,25.1
26300,2017-12-31 20:00:00,7.72,25.0
26301,2017-12-31 21:00:00,7.98,25.2
26302,2017-12-31 22:00:00,6.77,25.6


In [8]:
#se comprueba que si se hallan cambiando los valores
chequeo = dfwind['temperature'] >= 27
print(dfwind[chequeo].head(2))

                timestamp  wind_speed_ms-1  temperature
3688  2015-06-03 16:00:00              0.0         27.1
3689  2015-06-03 17:00:00              0.0         27.2
